In [ ]:
prompt = """Role: AI chatbot to help students
    Data: You will be provided with the whole information of the student about their name, age, gender, disability, language, pace of learning, interest
    Duty: By understading the person's condition you have to suggest them what should they learn at this age, help them to learn, suggest content, and answer question in a very simple manner
    Check: You have to check the person's knowledge about any topic and help them to understand the topic in a very simple manner. If you think they lack somewhere let them know.
    Output: Generate a structured response to help users understand the concepts clearly
    Language: Whatever the users feels good with you will get to know what language they are comfortable with
        """
temperature = .4
number_of_examples = 20

In [ ]:
!pip install google-generativeai

In [ ]:
import os
import random
import google.generativeai as genai
import time

genai.configure(api_key="YOUR_API_KEY")

def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 10:
            prev_examples = random.sample(prev_examples, 10)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    model = genai.GenerativeModel("gemini-1.5-flash")
    result = model.generate_content(messages[0].get('content'))
    print(result.text)
    return result.text

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)
    time.sleep(5)

print(prev_examples)

Generating example 0
```
prompt
-----------
Hi, my name is Alex, I'm 10 years old, male, no disability, I speak English, my learning pace is average, and I'm interested in space.  Can you suggest something I could learn about space?
-----------

response
-----------
Hi Alex! That's great you're interested in space! At your age, learning about the planets in our solar system would be a fun and engaging topic.  We could start with learning about the order of the planets from the sun, their sizes, and some interesting facts about each one.  Do you know the names of any planets?  If not, that's okay, we can learn them together!  I can also find you some cool videos and websites with pictures and information.
-----------
```

Generating example 1
```
prompt
-----------
Hi, my name is Alex, I'm 10 years old, male, no disability, I speak English, my learning pace is average, and I'm interested in space.  Can you suggest something I can learn about space?
-----------

response
-----------
Hi A

In [ ]:
import google.generativeai as genai

def generate_system_message(prompt):
    model = genai.GenerativeModel(
        "gemini-1.5-flash"
    )
    chat = model.start_chat(
        history=[
          {
            "role": "model",
            "parts": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "parts": prompt.strip(),
          }
        ]
    )
    repsonse = chat.send_message(prompt)
    return repsonse.text

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given student information (name, age, gender, disability, language, learning pace, interests), suggest appropriate learning materials, answer questions simply, check knowledge gaps, and provide structured explanations in the student's preferred language.
`. Feel free to re-run this cell if you want a better result.


In [ ]:
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.head()

There are 20 successfully-generated examples. Here are the first few:


,prompt,response
0,"Hi, my name is Alex, I'm 10 years old, male, n...",Hi Alex! That's great you're interested in spa...
1,"Hi, my name is Alex, I'm 10 years old, male, n...",Hi Alex! That's awesome you're interested in s...
2,"My name is Alex, I am 10 years old, male, no d...",Hi Alex! It's great you're interested in anim...
3,"Hi, I'm Alex, I'm 10 years old, male, no disab...",Hi Alex! That's great you're interested in spa...
4,"My name is Alex, I'm 10 years old, male, no di...",Hi Alex! That's a fantastic interest! At your ...


In [ ]:
# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# ... (Your other code remains the same)

# Check for GPU availability and set device accordingly
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Update device_map to use the selected device
# If using CPU, set device_map to "auto" to let Transformers choose the appropriate device
device_map = "auto" if device == torch.device("cpu") else {"": 0}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cur

In [ ]:
model_name = "NousResearch/llama-2-7b-chat-hf" # use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument
dataset_name = "train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

In [ ]:
# Load datasets
train_dataset = load_dataset('json', data_files='train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
)

# ... (Rest of your code)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
trainer.train()
trainer.model.save_pretrained(new_model)

# Cell 4: Test the model
logging.set_verbosity(logging.CRITICAL)
user_input = input("Ask anything >>> ")
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{user_input} [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

torch.cuda.empty_cache()

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
5,2.223400,1.920229


Ask anything >>> Hey I'm zade I'm 19 and I don't have any disability but i wanna learn coding


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[INST] <<SYS>>
Given student information (name, age, gender, disability, language, learning pace, interests), suggest appropriate learning materials, answer questions simply, check knowledge gaps, and provide structured explanations in the student's preferred language.

<</SYS>>

Hey I'm zade I'm 19 and I don't have any disability but i wanna learn coding [/INST]  Great to hear that you're interested in learning coding, Zade! 😊

Based on your information, I would recommend starting with some basic coding courses or tutorials that cover the basics of programming concepts, such as variables, data types, loops, and functions. These courses are usually designed for beginners and can help you build a solid foundation in coding.

Here are a few resources that you might find helpful:

1. Codecademy: Codecademy


In [ ]:
from transformers import pipeline, AutoTokenizer

name = input("Enter your name >>> ")
age = input("Enter your age >>> ")
gender = input("Enter your gender >>> ")
disability = input("Enter your disability >>> ")
language = input("Enter your language >>> ")
pace = input("Enter your pace >>> ")
interest = input("Enter your interest ( seperated by , ) >>> ")

print(f"Xfour: Hello {name}")

while True:
  user_input = input("( q to quit )>>> ")
  if user_input.lower == "q":
    break
  prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{user_input} [/INST]"
  num_new_tokens = 100
  num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

  max_length = num_prompt_tokens + num_new_tokens

  gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
  result = gen(prompt)
  print(result[0]['generated_text'].replace(prompt, ''))

torch.cuda.empty_cache()

KeyboardInterrupt: Interrupted by user